In [9]:
#Descomentar esto para istalar las librerias
# !pip install requests
# !pip install lxml
# !pip install selenium
# !pip install unidecode
# !pip install beautifulsoup4

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from unidecode import unidecode
import requests
import lxml as lxml
import html as html
import time
import pandas as pd
import numpy as np 
import re
import datetime

In [2]:
#Link a la pagina web
link = 'https://diaonline.supermercadosdia.com.ar/'
link_sin_barra = 'https://diaonline.supermercadosdia.com.ar'

In [3]:
# Inicializa el driver de Selenium (asegúrate de tener el driver correspondiente instalado)
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()

#Abre la página web en el navegador controlado por Selenium
driver.get(link)

#Hacemos click en el boton de categorias
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'diaio-custom-mega-menu-0-x-custom-mega-menu-trigger__button')))
button.click()

# Obtiene el HTML después de cargar todos los elementos
html_home = driver.page_source

# Cierra el navegador controlado por Selenium
driver.quit()

#Se busca dentro del HTML las categorias donde hacer hover
sp_home = BeautifulSoup(html_home, 'lxml')
sp_categorias = sp_home.find("div",{"class":"diaio-custom-mega-menu-0-x-category-list__container"}).find_all("a")
categorias = []
for cat in sp_categorias:
    # href = cat.get("href")
    categorias.append(cat.get("href"))
categorias 

['https://diaonline.supermercadosdia.com.ar/almacen',
 'https://diaonline.supermercadosdia.com.ar/bebidas',
 'https://diaonline.supermercadosdia.com.ar/frescos',
 'https://diaonline.supermercadosdia.com.ar/desayuno',
 'https://diaonline.supermercadosdia.com.ar/limpieza',
 'https://diaonline.supermercadosdia.com.ar/perfumeria',
 'https://diaonline.supermercadosdia.com.ar/congelados',
 'https://diaonline.supermercadosdia.com.ar/bebes-y-ninos',
 'https://diaonline.supermercadosdia.com.ar/hogar-y-deco',
 'https://diaonline.supermercadosdia.com.ar/mascotas',
 'https://diaonline.supermercadosdia.com.ar/almacen/golosinas-y-alfajores?PS=15&O=',
 'https://diaonline.supermercadosdia.com.ar/frescos/frutas-y-verduras',
 'https://diaonline.supermercadosdia.com.ar/electro-hogar']

In [4]:
#Primer categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
# Abre la página web en el navegador controlado por Selenium
driver.get(categorias[0])
html_categoria = driver.page_source

#Se obtiene la lista de categorias internas 
sp_categoria_interna = BeautifulSoup(html_categoria, 'lxml')
categorias_internas = sp_categoria_interna.find("div", {"class":"diaio-search-result-0-x-filter__container diaio-search-result-0-x-filter__container--plp bb b--muted-4 diaio-search-result-0-x-filter__container--category-2"})
lista_checkboxes = categorias_internas.find("div", {"class":"diaio-search-result-0-x-filterTemplateOverflow diaio-search-result-0-x-filterTemplateOverflow--plp pb5 overflow-y-auto"}).find_all("input", {"type":"checkbox"})

#Apretar boton pop-up
button_popup = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "vtex-modal__close-icon")))
button_popup.click()

#Lista de categorias dentro de la pagina
id_chechbox = []
for l in lista_checkboxes:
    id_chechbox.append(l.get("id"))
id_chechbox


#Obtencion del HTML que contiene todos los productos de una categoria
productos_obtenidos_html = {}

for i in id_chechbox:
    checkbox = driver.find_element(By.ID, i)
    time.sleep(1)
    checkbox.click()
    time.sleep(2)  # Activar el checkbox

    # Iniciar la altura inicial para el bucle
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Desplazarse un poco hacia abajo
        driver.execute_script("window.scrollBy(0, 1500);")
        time.sleep(2)  # Esperar para cargar más contenido

        # Obtener la nueva altura de la página
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Intentar hacer clic en el botón "Mostrar más"
        try:
            see_more_button = driver.find_element(By.XPATH, "//button[.//div[text()='Mostrar más']]")
            if see_more_button.is_displayed():
                # Desplazar el botón a la vista
                driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", see_more_button)
                time.sleep(1)
                # Verificar si está habilitado y hacer clic
                if see_more_button.is_enabled():
                    see_more_button.click()
                    print("Botón apretado")
                    time.sleep(2)  # Esperar después del clic
        except Exception as e:
            # Capturar el error pero seguir el bucle
            print(f"No se pudo hacer clic en el botón")

        # Salir del bucle si no hay más contenido
        if new_height == last_height:
            break

        # Actualizar la altura de la página para la siguiente iteración
        last_height = new_height

    # Regresar al inicio de la página

    time.sleep(5)
    html_productos = driver.page_source
    productos_obtenidos_html[i] = html_productos 

    driver.execute_script("window.scrollTo(0, 0);")

    # Deseleccionar el checkbox antes de pasar a la siguiente categoría
    time.sleep(2)
    checkbox.click()

driver.quit()


Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
Botón apretado
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
Botón apretado
Botón apretado
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
No se pudo hacer clic en el botón
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretad

In [ ]:


soup_3 = BeautifulSoup(productos_obtenidos_html['category-2-aceites-y-aderezos'], 'lxml')
productos_categoria = soup_3.find('div', {"id":"gallery-layout-container"}).find_all("div", {"class":"diaio-search-result-0-x-galleryItem"})

total_items = []

for litem_listado in range(len(productos_categoria)):
    try:
        # Extraemos link de la imagen del producto
        imagen = productos_categoria[litem_listado].find('img')
        link_imagen = imagen['src'] if imagen else None

        # Extraemos link a la web del producto en el supermercado
        web_producto = productos_categoria[litem_listado].find('a')
        url_producto = link_sin_barra + web_producto['href'] if web_producto else None

        # Datos del producto
        informacion_producto_wrapped = productos_categoria[litem_listado].find('div', 'vtex-flex-layout-0-x-flexCol')
        informacion_producto = informacion_producto_wrapped.find_all('div', 'vtex-flex-layout-0-x-flexColChild') if informacion_producto_wrapped else []

        # Creamos las listas soporte
        datos_productos_sin_procesar = []
        datos_productos_procesados = []
        # datos_productos_procesados.append(categorias[0])  # Agregamos la categoria a la que pertenece el producto
        # datos_productos_procesados.append(sub_categorias[s])  # Agregamos la categoria a la que pertenece el producto

        # Con este bucle for extraemos la informacion sin procesar
        for info in informacion_producto:
            valor_lista = info.get_text(strip=True).replace('\xa0', ' ')
            datos_productos_sin_procesar.append(valor_lista)

        # Generamos la expresion regular que extrae el precio correcto
        # y conformamos la lista con los datos del producto
        for i in range(0, 4):
            datos_productos_procesados.append(datos_productos_sin_procesar[i] if i < len(datos_productos_sin_procesar) else None)

        for i in range(len(datos_productos_sin_procesar)):
            patron_regex_precios = r"(\$\d{1,3}(?:\.\d{3})*,\d{2})"
            resultado_regex = re.findall(patron_regex_precios, datos_productos_sin_procesar[i])
            if resultado_regex:
                datos_productos_procesados.extend(resultado_regex)

        if len(datos_productos_procesados) == 7:
            datos_productos_procesados.append(None)
            datos_productos_procesados.append(datos_productos_sin_procesar[-1])
        else:
            datos_productos_procesados.append(datos_productos_sin_procesar[-1])

        datos_productos_procesados.append(url_producto)  # Agregamos el link al producto
        datos_productos_procesados.append(link_imagen)  # Agregamos el link a la imagen

        total_items.append(datos_productos_procesados)
    except Exception as e:
        print(f"Error procesando el producto {litem_listado}: {e}")

    lista_columnas = ('descuento', 'producto', 'campo_vacio_1', 'campo_vacio_2', 'precio_sin_descuento', 'precio', 'precio_por_k','link','imagen')
    dataframe_productos = pd.DataFrame(columns=lista_columnas)
    dataframe_productos['precio_sin_descuento'] = None
    dataframe_productos["fecha_scrapeo"] = datetime.date.today()
    
    csv_name = 'category-2-aceites-y-aderezos'
    dataframe_productos.to_csv(f'Productos Obtenidos/{csv_name}', index=False)


In [10]:
lista_columnas = ('descuento', 'producto', 'campo_vacio_1', 'campo_vacio_2', 'precio_sin_descuento', 'precio', 'precio_por_k','link','imagen')
dataframe_productos = pd.DataFrame(columns=lista_columnas, data=total_items)
dataframe_productos["fecha_scrapeo"] = datetime.date.today()
dataframe_productos.head()

,descuento,producto,campo_vacio_1,campo_vacio_2,precio_sin_descuento,precio,precio_por_k,link,imagen,fecha_scrapeo
0,-25%,Ketchup Clásico Hellmann's Doypack 250 Gr.,,,"$1.200,00","$900,00",4800.00KG,https://diaonline.supermercadosdia.com.ar/ketc...,https://ardiaprod.vtexassets.com/arquivos/ids/...,2024-12-06
1,-25%,Mostaza Original Doypack Savora 250 Gr.,,,"$880,00","$660,00",3520.00KG,https://diaonline.supermercadosdia.com.ar/most...,https://ardiaprod.vtexassets.com/arquivos/ids/...,2024-12-06
2,-25%,Mostaza Original Doypack Savora 500 Gr.,,,"$1.940,00","$1.455,00",3880.00KG,https://diaonline.supermercadosdia.com.ar/most...,https://ardiaprod.vtexassets.com/arquivos/ids/...,2024-12-06
3,-30%,Mayonesa De Ajo Hellmans 250 Gr,,,"$2.420,00","$1.694,00",7760.00KG,https://diaonline.supermercadosdia.com.ar/mayo...,https://ardiaprod.vtexassets.com/arquivos/ids/...,2024-12-06
4,-25%,Ketchup Hellmann's 500 Gr.,,,"$3.185,00","$2.388,75",6370.00KG,https://diaonline.supermercadosdia.com.ar/ketc...,https://ardiaprod.vtexassets.com/arquivos/ids/...,2024-12-06


In [14]:
productos_obtenidos_html.keys()

dict_keys(['category-2-aceites-y-aderezos', 'category-2-arroz-y-legumbres', 'category-2-comidas-listas', 'category-2-conservas', 'category-2-golosinas-y-alfajores', 'category-2-harinas', 'category-2-panaderia', 'category-2-pastas-secas', 'category-2-picadas', 'category-2-reposteria'])

In [15]:
for categ in productos_obtenidos_html.keys():
    soup_3 = BeautifulSoup(productos_obtenidos_html[categ], 'lxml')
    productos_categoria = soup_3.find('div', {"id":"gallery-layout-container"}).find_all("div", {"class":"diaio-search-result-0-x-galleryItem"})

    total_items = []  # Lista para acumular todos los productos

    for litem_listado in range(len(productos_categoria)):
        try:
            # Extracción link de la imagen del producto
            imagen = productos_categoria[litem_listado].find('img')
            link_imagen = imagen['src'] if imagen else None

            # Extracción link a la web del producto en el supermercado
            web_producto = productos_categoria[litem_listado].find('a')
            url_producto = link_sin_barra + web_producto['href'] if web_producto else None

            # Datos del producto
            informacion_producto_wrapped = productos_categoria[litem_listado].find('div', 'vtex-flex-layout-0-x-flexCol')
            informacion_producto = (
                informacion_producto_wrapped.find_all('div', 'vtex-flex-layout-0-x-flexColChild') 
                if informacion_producto_wrapped else []
            )

            # Creación listas para almacenar datos
            datos_productos_sin_procesar = []
            datos_productos_procesados = [None] * 9 

            # Extracción información sin procesar
            for info in informacion_producto:
                valor_lista = info.get_text(strip=True).replace('\xa0', ' ')
                datos_productos_sin_procesar.append(valor_lista)

            # Rellenar los datos procesados (posiciones fijas para evitar inconsistencias)
            for i in range(4):  # Copiar los primeros 4 valores (o rellenar con None si faltan)
                datos_productos_procesados[i] = (
                    datos_productos_sin_procesar[i] if i < len(datos_productos_sin_procesar) else None
                )

            # Buscar y añadir precios usando regex
            for i in range(len(datos_productos_sin_procesar)):
                patron_regex_precios = r"(\$\d{1,3}(?:\.\d{3})*,\d{2})"
                resultado_regex = re.findall(patron_regex_precios, datos_productos_sin_procesar[i])
                if resultado_regex:
                    datos_productos_procesados[4:4+len(resultado_regex)] = resultado_regex  # Coloca los precios en su lugar

            # Añadir último campo de información adicional si existe
            datos_productos_procesados[7] = url_producto  # Link del producto
            datos_productos_procesados[8] = link_imagen  # Link de la imagen

            # Agregar a la lista de productos totales
            total_items.append(datos_productos_procesados)

        except Exception as e:
            print(f"Error procesando el producto {litem_listado}: {e}")

        # Crear el DataFrame con columnas definidas
        lista_columnas = [
            'descuento', 'producto', 'campo_vacio_1', 'campo_vacio_2',
            'precio_sin_descuento', 'precio', 'precio_por_k',
            'link', 'imagen'
        ]
        dataframe_productos = pd.DataFrame(total_items, columns=lista_columnas)

        # Agregar la fecha de scrapeo como una columna adicional
        dataframe_productos["fecha_scrapeo"] = datetime.date.today()
        csv_name = categ
        dataframe_productos.to_csv(f'Productos Obtenidos/{csv_name}', index=False)


## Next steps

Poder profesionalizar este entorno para que cualquiera pueda acceder
